In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.0 MB/s eta 0:00:00


In [1]:
# ===================================================================
#  Library
# ===================================================================
import pandas as pd
from sklearn.metrics import mean_absolute_percentage_error

import warnings
warnings.simplefilter("ignore")

import numpy as np
from scipy.optimize import minimize
import optuna
import lightgbm as lgb

In [2]:
# ===================================================================
#  CFG
# ===================================================================
class CFG:
    seed = 42
    n_seeds = 3
    n_trials = 2000
    save_dir = "/content/drive/MyDrive/Colab Notebooks/signate2023/exp/"
    data_dir = "/content/drive/MyDrive/Colab Notebooks/signate2023/"
    filename = "exp00056"

In [3]:
# ===================================================================
#  Utils
# ===================================================================
def get_score(y_true, y_pred):
    """get MAPE score"""
    score = mean_absolute_percentage_error(y_true, y_pred)
    return score * 100

In [4]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/train.csv')
kun_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/kun_exp00052.csv')
kun_oof = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/kun_exp00052_oof_pred.csv')
yuji_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/exp050.csv')
yuji_oof = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/oof_df_exp050.csv')
df = pd.concat([kun_oof.merge(yuji_oof, on='id'), df_train['price']], axis=1)
test = kun_test.merge(yuji_test, on='id')

In [5]:
df

,id,kun_pred_0,kun_pred_1,kun_pred_2,kun_pred_3,kun_pred_4,kun_pred_5,kun_pred_6,kun_pred_7,kun_pred_8,...,pred_21,pred_22,pred_23,pred_24,pred_25,pred_26,pred_27,pred_28,pred_29,price
0,0,7118.9443,7627.1987,7084.0015,7043.0950,7122.7530,7335.2310,7350.0060,9836.8750,8781.6880,...,8613.396141,10717.468387,9447.344769,7984.506685,8272.276158,5876.657800,6016.322090,7374.013382,8496.804984,27587
1,1,3660.8042,3526.6816,3732.6855,3593.2502,3596.9211,3562.2925,3922.8992,3761.7544,3852.5068,...,4112.248835,3360.300264,3807.072052,3406.217022,3615.842705,4245.317355,4453.437876,4408.398403,3900.820201,4724
2,2,2883.7390,3149.7769,2995.7760,3038.0789,2876.8310,2982.5576,2787.5325,2911.1800,3172.8408,...,3167.730032,2929.953707,2957.126627,3618.763534,3022.052454,3913.586712,3705.560742,2921.102463,3116.785537,10931
3,3,8650.6270,8943.9030,9243.1750,8973.2610,9158.7180,8505.1610,8713.2310,8287.6740,8649.6400,...,7947.686753,8057.312440,8250.578545,8052.200549,8115.355025,8157.420044,7434.588221,8284.727211,7855.624335,16553
4,4,4031.1462,4406.5480,4107.9190,4059.3975,4229.2666,4429.8930,4209.0796,4026.7249,4228.1797,...,4200.673767,4535.020561,4420.531149,4421.192901,4254.960815,4031.139467,3972.143434,4027.789862,4720.410259,5158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27527,27527,12879.4610,13338.9980,12310.2180,12790.3090,12681.5170,12661.3610,11953.5220,12138.8550,12707.8850,...,12057.001640,13412.874272,12007.199573,12629.505062,11171.113895,10396.341985,14458.313885,14216.093476,12448.334050,32212
27528,27528,6693.9270,6920.0894,6770.7925,6634.9660,7131.3125,6796.1294,6812.7780,8508.7300,7977.3506,...,6344.288278,6026.474362,7055.987961,6138.980267,6987.163770,9398.511437,6896.820540,6121.882704,6568.329712,5400
27529,27529,14174.7040,15265.7090,14390.8955,14423.8730,12980.9630,14560.7450,14018.7240,13443.1000,13441.5670,...,16020.269380,14682.474553,14787.659188,13376.032925,13611.897758,14921.987650,15042.231812,14166.439728,15120.009430,22227
27530,27530,7086.4136,6367.2173,6483.3950,7116.9200,7356.8930,7240.1910,7432.3013,7450.1953,7293.5034,...,6997.856606,8323.489037,6872.441408,6424.874952,6747.818100,7189.621305,7638.963008,6865.697187,6743.532957,3054


In [6]:
test

,id,kun_pred_0,kun_pred_1,kun_pred_2,kun_pred_3,kun_pred_4,kun_pred_5,kun_pred_6,kun_pred_7,kun_pred_8,...,pred_20,pred_21,pred_22,pred_23,pred_24,pred_25,pred_26,pred_27,pred_28,pred_29
0,27532,9462.117521,9616.369529,9455.621674,9411.321320,9439.296013,9339.256741,9496.255924,9169.724825,9408.697900,...,8916.874560,9138.275094,9312.076577,9211.745829,9498.316353,8687.940060,9208.293004,8356.975825,9330.770219,9692.941277
1,27533,5431.933504,5579.297671,5502.604659,5421.178000,5426.123259,5696.500918,5520.123463,5516.627145,5408.202246,...,5508.598523,5524.770227,5759.403447,5638.017983,5628.653785,6202.828111,4785.541393,5604.598269,5398.402382,5744.756995
2,27534,5563.460291,5543.298528,5644.606929,5568.560278,5470.429184,5637.070499,5705.703291,5364.698823,5388.325417,...,5506.078552,5390.600874,5432.630177,5378.844623,5732.867996,5793.773542,5632.035892,5905.933910,5561.372542,5972.185238
3,27535,19615.535011,19333.748566,19484.368965,19209.276232,19294.428362,19318.374103,19171.163948,16937.493371,17264.432164,...,20360.798126,19146.682299,17090.358841,18263.872975,17604.417956,17961.093949,12794.552170,20108.010799,19183.244173,19561.667694
4,27536,4165.521012,4132.487296,4262.280629,4212.731834,4118.160390,4143.661658,4292.907859,4212.163549,4187.348948,...,4550.212166,3989.678345,5322.743324,4245.195615,3830.999498,3871.618438,6658.960126,4595.955768,4849.609894,4540.888135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27532,55064,13342.552114,13667.855920,13413.909141,13440.577192,13313.740622,12874.275284,13745.622717,14408.987795,14279.202379,...,9823.251404,13452.186317,12280.179966,10014.768978,13841.399729,12156.012675,11113.945897,13684.552604,13453.234406,12058.316829
27533,55065,8482.824014,8544.580408,8316.495493,8741.763658,8683.826037,8274.076872,8320.918352,9910.438599,10146.887679,...,11215.002932,9608.711346,9416.906835,8761.781589,9408.583043,9972.958526,10205.926610,8689.446574,10127.392831,8915.794126
27534,55066,6235.427898,6279.775623,6330.773011,6197.646469,6290.777000,6289.493699,6198.501766,6305.310528,6316.055393,...,5307.977447,5567.878736,5442.195710,6072.441981,5394.629396,5613.617670,6456.335598,5536.471253,5446.296301,5800.008342
27535,55067,5198.181210,5218.070387,5204.748284,5161.087157,5189.810533,5238.875991,5187.003192,4991.335690,5103.491682,...,4927.390509,5421.294054,5053.042998,5146.113453,4919.544215,5329.443448,4729.350399,5490.082143,4927.573842,4721.779342


In [7]:
def get_custom_cv(df, n_splits=5):
    # priceを小さい順に各foldに振り分ける
    df = df.sort_values(by="price", ignore_index=True)
    df["fold"] = [i for i in range(n_splits)] * (df.shape[0] // n_splits) \
                + [i for i in range(df.shape[0] % n_splits)]
    df = df.sort_values(by="id", ignore_index=True)


    for fold in range(n_splits):
        train_idx = df[df["fold"] != fold].index
        valid_idx = df[df["fold"] == fold].index
        yield train_idx, valid_idx

n_splits = 8
cv = list(get_custom_cv(df, n_splits))

In [14]:
def fit_lgbm(X, y, test_data, cv, categorical_cols: list = None, params: dict = None, verbose: int = 50):
    if params is None:
        params = {}

    models = []
    n_records = len(X)
    oof_pred = np.zeros((n_records,), dtype=np.float32)
    test_list = []

    for i, (idx_train, idx_valid) in enumerate(cv):
        X_tr, y_tr = X.iloc[idx_train], y.iloc[idx_train]
        X_val, y_val = X.iloc[idx_valid], y.iloc[idx_valid]
        transformed_test_data = test_data.copy()


        drop_cols = ['id', 'price']
        X_tr.drop(drop_cols, axis=1, inplace=True)
        X_val.drop(drop_cols, axis=1, inplace=True)
        transformed_test_data.drop(['id'], axis=1, inplace=True)
        test_list.append(transformed_test_data)


        model = lgb.LGBMRegressor(**params)
        model.fit(
            X_tr,
            y_tr,
            categorical_feature=categorical_cols,
            eval_set=[(X_val, y_val)],
            callbacks=[lgb.early_stopping(100, verbose=verbose)],
        )
        pred_i = model.predict(X_val)
        oof_pred[idx_valid] = pred_i
        models.append(model)
        score = mean_absolute_percentage_error(y_val, pred_i)
        print(f" - fold{i + 1} - {score:.4f}")

    score = mean_absolute_percentage_error(y, oof_pred)
    print("=" * 50)
    print(f"FINISHI: Whole Score: {score:.4f}")
    return score, oof_pred, models, test_list




# optunaを使ったパラメータチューニング
def tuning(df, cv):
     def objective(trial):
         max_depth = trial.suggest_int("max_depth", 1, 10)
         params = {
              "max_depth": max_depth,
              "n_estimators": trial.suggest_int('n_estimators', 100, 50000),
              "learning_rate": trial.suggest_float("learning_rate", 0.01, 1.0),
              "num_leaves": trial.suggest_int("num_leaves", 2, 2**max_depth),
              "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.1, 1.0),
              "subsample": trial.suggest_float("subsample", 0.01, 1.0),
              "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 0, 300),
              "max_bin": trial.suggest_int("max_bin", 32, 512),
              "reg_lambda": trial.suggest_float("reg_lambda", 1e-10, 100.0),
              "task":"train",
              "objective": "mape",
              "boosting":"gbdt",
              "metric": 'mape',
              "importance_type": "gain",
              "seed" : 88,
              "verbosity": -1,


         }

         score, _, _, _ = fit_lgbm(
             X=df, y=df["price"], test_data=test, cv=cv,  categorical_cols=[], params=params, verbose=-1
         )
         return score

     study = optuna.create_study(direction="minimize")
     study.optimize(objective, n_trials=100)
     print("Number of finished trials:", len(study.trials))
     print("Best trial:", study.best_trial.params)
     return study.best_trial.params

In [ ]:
lgbm_params = tuning(df, cv)

lgbm_params.update(
    {
    "task":"train",
    "objective": "mape",
    "boosting":"gbdt",
    "metric": 'mape',
    "importance_type": "gain",
    "seed" : 88,
    "verbosity": -1,
    }
)


# training
score, oof_lgb, models_lgb, transformed_test_data_list = fit_lgbm(
    X=df,
    y=df["price"],
    test_data=test,
    categorical_cols=[],
    params=lgbm_params,
    cv=cv,
    verbose=False,
)

In [15]:
lgbm_params = {
    "task":"train",
    "objective": "mape",
    "boosting":"gbdt",
    "metric": 'mape',
    "seed" : 88,
    "importance_type": "gain",
    "random_state": 88,
    "verbose": -1,
    'max_depth': 4,
    'n_estimators': 29169,
    'learning_rate': 0.05877354562849153,
    'num_leaves': 6,
    'colsample_bytree': 0.21666232965179164,
    'subsample': 0.8854776877757361,
    'min_data_in_leaf': 198,
    'max_bin': 46,
    'reg_lambda': 99.69138915921155

}


# training
score, oof_lgb, models_lgb, transformed_test_data_list = fit_lgbm(
    X=df,
    y=df["price"],
    test_data=test,
    categorical_cols=[],
    params=lgbm_params,
    cv=cv,
    verbose=False,
)

 - fold1 - 0.4319
 - fold2 - 0.4427
 - fold3 - 0.4332
 - fold4 - 0.4448
 - fold5 - 0.4286
 - fold6 - 0.4277
 - fold7 - 0.4442
 - fold8 - 0.4353
FINISHI: Whole Score: 0.4360


In [ ]:
Best trial: {'max_depth': 4, 'n_estimators': 29169, 'learning_rate': 0.05877354562849153, 'num_leaves': 6, 'colsample_bytree': 0.21666232965179164, 'subsample': 0.8854776877757361, 'min_data_in_leaf': 198, 'max_bin': 46, 'reg_lambda': 99.69138915921155}
 - fold1 - 0.4319
 - fold2 - 0.4427
 - fold3 - 0.4332
 - fold4 - 0.4448
 - fold5 - 0.4286
 - fold6 - 0.4277
 - fold7 - 0.4442
 - fold8 - 0.4353
==================================================
FINISHI: Whole Score: 0.4360

In [16]:
sub_df = pd.read_csv(CFG.data_dir + "submit_sample.csv", names=["id", "price"])

In [18]:
preds = []
for transformed_test_data, model in zip(transformed_test_data_list, models_lgb):
    pred = model.predict(transformed_test_data)
    preds.append(pred)
pred = np.mean(preds, axis=0)

# submission
sub_df["price"] = pred
sub_df.to_csv(f'/content/drive/MyDrive/Colab Notebooks/signate2023/exp/kun_{CFG.filename}.csv', index=False, header=False)

In [20]:
pd.concat([df['id'], pd.DataFrame(oof_lgb, columns=['oof_pred'])], axis=1).to_csv(f'/content/drive/MyDrive/Colab Notebooks/signate2023/exp/kun_{CFG.filename}_oof_pred.csv', index=False)